In [ ]:
# edit line no. 14
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import ast
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 1. Data Preprocessing

# Load CSV file
df = pd.read_csv('a.csv')

# Parse the input string data
df['inputs'] = df['inputs'].apply(lambda x: ast.literal_eval(x))

# Convert the labels to numerical values using LabelEncoder
label_encoder = LabelEncoder()
df['labels'] = label_encoder.fit_transform(df['labels'])

# Prepare input and output arrays
X = torch.tensor(df['inputs'].tolist(), dtype=torch.float32)
y = torch.tensor(df['labels'].values, dtype=torch.long)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 2. Neural Network Design

class ThreeLayerNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ThreeLayerNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

# Set the input, hidden, and output dimensions
input_size = X_train.shape[1]
hidden_size = 128  # Can be tuned
output_size = len(label_encoder.classes_)  # 8 classes

# Instantiate the model
model = ThreeLayerNN(input_size, hidden_size, output_size)

# 3. Training

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training the model
num_epochs = 100  # Can be tuned
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 4. Evaluation

# Predict on test data
with torch.no_grad():
    y_pred = model(X_test)
    _, predicted = torch.max(y_pred.data, 1)
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, predicted)
    print(f'Accuracy: {accuracy * 100:.2f}%')

# 5. Make prediction on new input data (example)
def classify_input(input_data):
    input_tensor = torch.tensor(input_data, dtype=torch.float32)
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted_class = torch.max(output.data, 0)
    return label_encoder.inverse_transform([predicted_class.item()])[0]


Epoch [10/100], Loss: 2.0094
Epoch [20/100], Loss: 1.8803
Epoch [30/100], Loss: 1.6708
Epoch [40/100], Loss: 1.3665
Epoch [50/100], Loss: 1.1490
Epoch [60/100], Loss: 1.1243
Epoch [70/100], Loss: 1.0998
Epoch [80/100], Loss: 1.0937
Epoch [90/100], Loss: 1.0892
Epoch [100/100], Loss: 1.0875
Accuracy: 70.56%
